# Qwen3: Supervised Fine-Tuning with TRL

This notebook demonstrates how to fine-tune a language model using the Supervised Fine-Tuning (SFT) approach with the TRL library.

## Install required libraries

In [ ]:
!pip install -U transformers torch torchaudio torchvision trl peft bitsandbytes accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wandb login

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: woosh98 (woosh98-inha-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



## Import libraries

In [ ]:
import torch
from datasets import load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import DPOConfig, DPOTrainer
from peft import LoraConfig

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: /content/prepared_finetuning_dataset_splits_arrow_dpo.zip 이 압축파일을 푸는 코드를 작성해 줘

!unzip /content/prepared_finetuning_dataset_splits_arrow_dpo.zip -d /content/

Archive:  /content/prepared_finetuning_dataset_splits_arrow_dpo.zip
   creating: /content/prepared_finetuning_dataset_splits_arrow_dpo/
  inflating: /content/prepared_finetuning_dataset_splits_arrow_dpo/dataset_dict.json  
   creating: /content/prepared_finetuning_dataset_splits_arrow_dpo/test/
  inflating: /content/prepared_finetuning_dataset_splits_arrow_dpo/test/data-00000-of-00001.arrow  
  inflating: /content/prepared_finetuning_dataset_splits_arrow_dpo/test/dataset_info.json  
  inflating: /content/prepared_finetuning_dataset_splits_arrow_dpo/test/state.json  
   creating: /content/prepared_finetuning_dataset_splits_arrow_dpo/train/
  inflating: /content/prepared_finetuning_dataset_splits_arrow_dpo/train/data-00000-of-00001.arrow  
  inflating: /content/prepared_finetuning_dataset_splits_arrow_dpo/train/dataset_info.json  
  inflating: /content/prepared_finetuning_dataset_splits_arrow_dpo/train/state.json  
   creating: /content/prepared_finetuning_dataset_splits_arrow_dpo/valida

# Load Dataset

In [ ]:
dataset_path = "prepared_finetuning_dataset_splits_arrow_dpo"

# Arrow 형식의 데이터셋 로드
dataset = load_from_disk(dataset_path)

train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print("데이터셋 로드 완료!")
print(f"학습 데이터 개수: {len(train_dataset)}")
print(f"검증 데이터 개수: {len(eval_dataset)}")

데이터셋 로드 완료!
학습 데이터 개수: 610
검증 데이터 개수: 305


## Configuration

3. 모델 및 토크나이저 설정 (QLoRA 적용)
A100 GPU의 메모리를 효율적으로 사용하기 위해, 4비트 양자화(Quantization)와 LoRA(Low-Rank Adaptation)를 함께 사용하는 QLoRA 방식으로 모델을 로드합니다.

In [ ]:
# Model configuration
model_name = "Qwen/Qwen3-4B"  # You can change this to any model you want to fine-tune

# LoRA 설정 (PEFT)
# 검증 결과, 아래의 target_modules가 Qwen2 아키텍처에 가장 적합합니다.
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    task_type="CAUSAL_LM",
)

# 4비트 QLoRA 양자화 설정
# A100 GPU는 bfloat16을 지원하므로, 이를 compute dtype으로 설정하는 것이 가장 효율적입니다.
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# 모델 로딩
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

print("검증된 설정으로 모델과 토크나이저 로딩 완료!")

# Training configuration
use_peft = True  # Set to True to use Parameter-Efficient Fine-Tuning (PEFT)
output_dir = "./output/sft-model"
num_train_epochs = 1
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
learning_rate = 2e-4 if use_peft else 2e-5  # Higher learning rate for PEFT

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/32.8k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

검증된 설정으로 모델과 토크나이저 로딩 완료!


## Load model and tokenizer

In [ ]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    use_cache=False,  # Disable KV cache during training
    device_map="auto"
)

# ↓ 여기에 SFT 어댑터 로드/적용 부분을 추가 ↓
from peft import PeftModel

sft_model = PeftModel.from_pretrained(
    model,
    "totalcream/Qwen3-4B-finetunning-Adapter",  # Hugging Face에 올려둔 SFT 어댑터 repo ID
    device_map="auto",
    is_trainable=True,                       # DPO 단계에서 업데이트하려면 True
)
model = sft_model  # 이후 DPOTrainer에는 이 model을 넘깁니다

# (원래 있던) 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# # Set up chat formatting (if the model doesn't have a chat template)
# if tokenizer.chat_template is None:
#     model, tokenizer = setup_chat_format(model, tokenizer, format="chatml")

# # Set padding token
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/845 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/66.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
# DPO 학습을 위한 최종 하이퍼파라미터 설정
training_args = DPOConfig(
    output_dir="./qwen3-dpo-final-results", # 결과 저장 디렉토리
    num_train_epochs=2,                   # 데이터셋이 610개이므로 2~3 에포크가 적당합니다.
    per_device_train_batch_size=2,        # A100 메모리를 고려한 안정적인 배치 사이즈
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,        # 유효 배치 사이즈를 16 (2*8)으로 설정하여 학습 안정화
    gradient_checkpointing=True,          # 메모리 사용량 최적화
    optim="paged_adamw_8bit",             # 메모리 효율적 옵티마이저

    logging_steps=10,                     # 10 스텝마다 학습 로그 출력
    save_steps=50,                        # 50 스텝마다 모델 어댑터 저장
    eval_steps=50,                        # 50 스텝마다 검증 평가 실행

    learning_rate=5e-7,                   # DPO의 핵심: 검증된 안정적인 낮은 학습률
    beta=0.1,                             # DPO 손실 강도, 0.1이 표준적인 시작점

    lr_scheduler_type="linear",           # 선형 학습률 스케줄러
    warmup_ratio=0.1,                     # 학습 초반에 learning rate를 서서히 증가시켜 안정성 확보

    report_to="wandb",                    # Weights & Biases 로깅 활성화
    run_name="qwen3-4b-dpo-custom-data",  # WandB에 표시될 실행 이름
)

print("검증된 하이퍼파라미터로 DPO 학습 설정 완료!")

검증된 하이퍼파라미터로 DPO 학습 설정 완료!


In [ ]:
# DPOTrainer 초기화
# DPOTrainer는 tokenizer 객체를 'processing_class'라는 이름의 인자로 받습니다.
trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    processing_class=tokenizer  # <-- 이 부분이 수정되었습니다!
)

# DPO 학습 시작
print("Qwen3 모델 DPO 파인튜닝을 시작합니다...")
trainer.train()
print("DPO 학습이 성공적으로 완료되었습니다!")

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Extracting prompt in train dataset:   0%|          | 0/610 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/610 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/610 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/305 [00:00<?, ? examples/s]

...

wandb: Currently logged in as: woosh98 (woosh98-inha-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.699600
20,0.696900
30,0.712900
40,0.658600
50,0.687800
60,0.704100
70,0.675900


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


DPO 학습이 성공적으로 완료되었습니다!


## Save the fine-tuned model

In [ ]:
# 최종 모델(LoRA 어댑터) 저장
final_adapter_path = "./qwen3-dpo-final-adapter"
trainer.save_model(final_adapter_path)

print(f"학습된 LoRA 어댑터가 '{final_adapter_path}' 경로에 저장되었습니다.")
print("\n--- 학습된 모델 테스트 방법 예시 ---")
print("1. 원본 Qwen2 모델을 로드합니다.")
print("2. PeftModel.from_pretrained()를 사용해 저장된 어댑터를 적용합니다.")
print("3. 병합된 모델로 추론을 수행합니다.")

# 아래는 나중에 모델을 불러와 테스트할 때 사용할 수 있는 예시 코드입니다.
# from peft import PeftModel
#
# # 1. 원본 모델 다시 로드 (bfloat16으로)
# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
#
# # 2. 학습된 LoRA 어댑터와 병합
# tuned_model = PeftModel.from_pretrained(base_model, final_adapter_path)
#
# # 3. 추론 수행
# prompt = "여기에 테스트할 프롬프트를 입력하세요."
# inputs = tokenizer(prompt, return_tensors="pt").to(tuned_model.device)
# outputs = tuned_model.generate(**inputs, max_new_tokens=100)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

학습된 LoRA 어댑터가 './qwen3-dpo-final-adapter' 경로에 저장되었습니다.

--- 학습된 모델 테스트 방법 예시 ---
1. 원본 Qwen2 모델을 로드합니다.
2. PeftModel.from_pretrained()를 사용해 저장된 어댑터를 적용합니다.
3. 병합된 모델로 추론을 수행합니다.


In [ ]:
# prompt: SFT 어뎁터를 저장하는 코드를 작성해 줘

# Construct the save path in Google Drive
drive_save_path = "output/Adapter"
# Ensure the directory exists
os.makedirs(drive_save_path, exist_ok=True)
# Save the adapter model
trainer.model.save_pretrained(drive_save_path)
print(f"Adapter model saved to: {drive_save_path}")

Adapter model saved to: output/Adapter


In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import PeftModel
from huggingface_hub import HfApi, login
from google.colab import userdata
import os

# --- 0) bitsandbytes 설치 (한번만) ---
# !pip install bitsandbytes

# --- 설정 ---
hf_token = userdata.get('HF_TOKEN')
login(token=hf_token)
api = HfApi()

model_name = "Qwen/Qwen3-4B"
sft_adapter_repo = "totalcream/Qwen3-4B-finetunning-Adapter"
final_dpo_adapter_path = "./qwen3-dpo-final-adapter"
hf_repo_name_merged = "totalcream/Qwen3-4B-SFT-DPO-Merged"
local_save_path = "./qwen3-sft-dpo-merged-final"

# --- 1) 4-bit 양자화 설정 ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,              # 4bit 로드
    bnb_4bit_quant_type="nf4",      # NF4 양자화
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# --- 2) Base 모델 로드 (4-bit) ---
print("1) 4-bit 양자화된 베이스 모델 로드")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# --- 3) SFT 어댑터 적용 → 병합 ---
print("2) SFT 어댑터 적용 및 병합")
sft_model = PeftModel.from_pretrained(base_model, sft_adapter_repo)
sft_merged = sft_model.merge_and_unload()  # → 순수 AutoModelForCausalLM (4-bit 상태 유지)

# --- 4) DPO 어댑터 적용 → 병합 ---
print("3) DPO 어댑터 적용 및 병합")
dpo_model = PeftModel.from_pretrained(sft_merged, final_dpo_adapter_path)
final_model = dpo_model.merge_and_unload()  # → 순수 AutoModelForCausalLM (역시 4-bit)

# --- 5) 최종 모델 저장 (.safetensors) ---
print(f"4) 최종 모델 저장: {local_save_path}")
os.makedirs(local_save_path, exist_ok=True)
final_model.save_pretrained(
    local_save_path,
    safe_serialization=True  # → model.safetensors 생성
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(local_save_path)

# --- 6) 허브에 업로드 ---
print(f"5) 허브에 업로드: {hf_repo_name_merged}")
api.create_repo(hf_repo_name_merged, repo_type="model", exist_ok=True)
api.upload_folder(
    folder_path=local_save_path,
    repo_id=hf_repo_name_merged,
    repo_type="model",
    commit_message="Upload 4bit-quantized SFT+DPO merged model"
)

print("🎉 완료! 허브에서 양자화된 model.safetensors 확인하세요.")


1) 4-bit 양자화된 베이스 모델 로드


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2) SFT 어댑터 적용 및 병합


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


3) DPO 어댑터 적용 및 병합
4) 최종 모델 저장: ./qwen3-sft-dpo-merged-final
5) 허브에 업로드: totalcream/Qwen3-4B-SFT-DPO-Merged


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

🎉 완료! 허브에서 양자화된 model.safetensors 확인하세요.


In [ ]:
model.push_to_hub("totalcream/Qwen3-4B-DPO-model")

adapter_model.safetensors:   0%|          | 0.00/66.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/totalcream/Qwen3-4B-DPO-model/commit/aad33aa7dbbf9c0c1f31a52ef69db5157fd35297', commit_message='Upload model', commit_description='', oid='aad33aa7dbbf9c0c1f31a52ef69db5157fd35297', pr_url=None, repo_url=RepoUrl('https://huggingface.co/totalcream/Qwen3-4B-DPO-model', endpoint='https://huggingface.co', repo_type='model', repo_id='totalcream/Qwen3-4B-DPO-model'), pr_revision=None, pr_num=None)

## Test the fine-tuned model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# --- 모델 및 토크나이저 경로 설정 ---
model_name = "Qwen/Qwen3-4B"
adapter_path = "./qwen3-dpo-final-adapter" # DPO 학습 후 저장한 어댑터 경로

# --- 토크나이저 로딩 ---
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# --- 모델 로딩 ---
# 1. 원본 베이스 모델 로딩
original_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# 2. DPO 튜닝된 모델 로딩 (베이스 모델 + LoRA 어댑터)
tuned_model = PeftModel.from_pretrained(original_model, adapter_path)

print("원본 모델과 DPO 튜닝 모델 로딩 완료!")

# 두 모델 모두 추론 모드로 설정
original_model.eval()
tuned_model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

원본 모델과 DPO 튜닝 모델 로딩 완료!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(


In [ ]:
# --- 테스트 프롬프트 정의 ---
prompt = """
당신은 전기전자공학부 전문가입니다. 회로이론에 관해서 객관식 문제를 하나 생성하세요. 생성할 내용은 다음과 같습니다.

질문:
보기1:
보기2:
보기3:
보기4:
보기5:
정답:
해설:

반드시 '해설' 부분까지 모두 포함하여 위의 형식에 맞춰 전체 문제를 완성해 주세요.
"""

# --- Qwen2 채팅 템플릿 적용 ---
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
formatted_prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(original_model.device)

# --- 응답 생성 파라미터 ---
generation_args = {
    "max_new_tokens": 2048,
    "temperature": 0.7,
    "top_p": 0.9,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


# ==========================================================
# 1. 원본 모델 응답 생성
# ==========================================================
print("✅ 1. 원본 Qwen3 모델의 응답")
print("==========================================================")
with torch.no_grad():
    original_outputs = original_model.generate(**inputs, **generation_args)
# 프롬프트를 제외한 순수 응답만 디코딩
original_response = tokenizer.decode(original_outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
print(original_response)


# ==========================================================
# 2. DPO 튜닝된 모델 응답 생성
# ==========================================================
print("\n\n✅ 2. DPO 튜닝된 모델의 응답")
print("==========================================================")
with torch.no_grad():
    tuned_outputs = tuned_model.generate(**inputs, **generation_args)
# 프롬프트를 제외한 순수 응답만 디코딩
tuned_response = tokenizer.decode(tuned_outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True)
print(tuned_response)

✅ 1. 원본 Qwen3 모델의 응답
<think>
Okay, the user wants me to create an objective question about circuit theory for an electrical engineering student. Let me start by recalling some key concepts in circuit theory. Maybe something related to Kirchhoff's laws or Ohm's law? Oh, Kirchhoff's Voltage Law (KVL) is a good topic. It states that the sum of all voltages around a closed loop is zero.

I need to make sure the question is clear and the answer is unambiguous. Let me think of a scenario where KVL is applied. For example, a simple loop with a battery and two resistors. The question could ask for the voltage drop across a resistor. 

Wait, the options should be different but plausible. Let me set up the problem. Suppose a loop with a 12V battery, a 2Ω resistor, and a 4Ω resistor. The total resistance is 6Ω, so the current would be 2A. Then the voltage drop across the 2Ω resistor is 4V, and across the 4Ω resistor is 8V. 

But the question should be phrased as a multiple-choice. Maybe ask which

In [ ]:
model.push_to_hub("burtenshaw/Qwen3-30B-A3B-python-code")

In [ ]:
from huggingface_hub import notebook_login

# 허깅페이스 로그인 창이 뜨면 토큰을 입력합니다.
notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# --- 경로 및 ID 설정 ---
# 1. 모든 것의 시작점, 원본 베이스 모델
base_model_name = "Qwen/Qwen3-4B"

# 2. 1차로 튜닝했던 SFT 모델(어댑터)의 허브 ID
my_sft_model_hub_id = "totalcream/업로드한-모델-이름" # 본인 ID로 변경!

# 3. 2차로 튜닝한 최종 DPO 어댑터의 로컬 경로
final_dpo_adapter_path = "./my-final-dpo-adapter" # 이전 단계에서 저장한 경로

# --- 모델 로딩 및 순차적 병합 ---
print("모델 병합을 시작합니다...")

# 1. 원본 베이스 모델을 bfloat16으로, 양자화 없이 전체 로드합니다.
#    (병합을 위해서는 전체 모델이 메모리에 올라와야 합니다)
print("1단계: 원본 베이스 모델 로딩...")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# 2. 베이스 모델 위에 SFT 어댑터를 먼저 적용합니다.
print("2단계: SFT 어댑터 적용...")
sft_model = PeftModel.from_pretrained(base_model, my_sft_model_hub_id)

# 3. SFT 모델 위에 최종 DPO 어댑터를 적용합니다.
print("3단계: 최종 DPO 어댑터 적용...")
dpo_model = PeftModel.from_pretrained(sft_model, final_dpo_adapter_path)


# 4. 모든 어댑터의 가중치를 베이스 모델에 완전히 통합합니다.
print("4단계: 모든 어댑터 가중치 병합...")
merged_model = dpo_model.merge_and_unload()

print("✅ 모든 모델 병합 완료!")

# 토크나이저도 로드합니다 (업로드 시 필요)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token